In [1]:
import cv2
import numpy as np
from darkflow.net.build import TFNet
from filterpy.kalman import KalmanFilter
from sklearn.utils.linear_assignment_ import linear_assignment
from numba import jit

## Import YOLO

I am using a neural network called YOLO (you only look once), wich is amongst the state-of-the-art CNN's for object detection and classification. In contrast to handcrafted algorithms that detect shapes and features, CNN's can generalize better when trained on a sufficient amount of data, because they figure out the best features to detect and classify objects on their own through back propagation.
By loading the weights of the network I don't have to train it myself and can profit from the training that has been done on a large database. It would be possible to retrain the last layers for specific tasks but in this case its not necessary, because it already detects the classes I am looking for.

In [2]:
options = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.3,
    'gpu': 0.8
}

tfnet = TFNet(options)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.020043373107910156s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep

In [3]:
class frame():
    def __init__(self):
        self.input_img = [np.array([False])]
        self.result = [np.array([False])]
        self.detections = None
        self.output_img = [np.array([False])]

# function to filter the detections to class "car" only and obtain the bounding boxes in format for drawing them on the image
def get_YOLO_detections(frame):
    if frame.result:
        detections = []
        for entry in frame.result:
            if entry['label'] == 'car':
                detection = [entry['topleft']['x'], entry['topleft']['y'], entry['bottomright']['x'], entry['bottomright']['y'], entry['confidence']]
                detections.append(detection)
    else:
        detections = []
    frame.detections = np.array(detections)

# function to draw the bounding boxes on the image
def draw_detections(frame, thick=6):
    draw_img = np.copy(frame.input_img)
    #[cv2.rectangle(draw_img, (box[0][0], box[0][1]), (box[1][0], box[1][1]), color, thick) for box in frame.boxes]
    colors = ((0, 255, 255), (0, 100, 255), (0, 0, 255), (100, 0, 255), (255, 0, 255), (255, 0, 100), (255, 0, 0), (255, 100, 0), (255, 255, 0), (100, 255, 0), (0, 255, 0))
    #print(int(frame.detections_filt[0,-1]%len(colors)))
    [cv2.rectangle(draw_img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), colors[int(box[-1]%len(colors))], thick) for box in frame.detections_filt]
    #[cv2.circle(draw_img, center, 10, color, -1) for center in frame.centers]
    frame.output_img = draw_img

def update_detection_filter(frame, tracker):
    #print(frame.detections)
    frame.detections_filt = tracker.update(frame.detections)
    #print(frame.detections_filt)

## Tracker
(imported from github repo)
I am using a kalman filter for each individual detection to estimate the detection position in the next frame. In the update step the kalman gain decides the weights of prediction and measurement to update the estimate.

The state contains:

* center point x
* center point y
* area size (w\*h)
* aspect ratio (w/h)
* velocity x
* velocity y

And is calculated for each object.

In [12]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016 Alex Bewley alex@dynamicdetection.com

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

@jit
def iou(bb_test,bb_gt):
    """
    Computes IUO between two bboxes in the form [x1,y1,x2,y2]
    """
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1]) + (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1]) - wh)
    return(o)

def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
    """
    w = bbox[2]-bbox[0]
    h = bbox[3]-bbox[1]
    x = bbox[0]+w/2.
    y = bbox[1]+h/2.
    s = w*h    #scale is just area
    r = w/float(h)
    return np.array([x,y,s,r]).reshape((4,1))

def convert_x_to_bbox(x,score=None):
    """
    Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
    """
    w = np.sqrt(x[2]*x[3])
    h = x[2]/w
    if(score==None):
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
    else:
        return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
    """
    This class represents the internel state of individual tracked objects observed as bbox.
    """
    count = 0
    def __init__(self,bbox):
        """
        Initialises a tracker using initial bounding box.
        """
        #define constant velocity model
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])
    
        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01
    
        self.kf.x[:4] = convert_bbox_to_z(bbox)
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self,bbox):
        """
        Updates the state vector with observed bbox.
        """
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(convert_bbox_to_z(bbox))

    def predict(self):
        """
        Advances the state vector and returns the predicted bounding box estimate.
        """
        if((self.kf.x[6]+self.kf.x[2])<=0):
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if(self.time_since_update>0):
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(convert_x_to_bbox(self.kf.x))
        return self.history[-1]

    def get_state(self):
        """
        Returns the current bounding box estimate.
        """
        return convert_x_to_bbox(self.kf.x)

def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
    """
    Assigns detections to tracked object (both represented as bounding boxes)

    Returns 3 lists of matches, unmatched_detections and unmatched_trackers
    """
    if(len(trackers)==0):
        return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
    iou_matrix = np.zeros((len(detections),len(trackers)),dtype=np.float32)

    for d,det in enumerate(detections):
        for t,trk in enumerate(trackers):
            iou_matrix[d,t] = iou(det,trk)
    matched_indices = linear_assignment(-iou_matrix)

    unmatched_detections = []
    for d,det in enumerate(detections):
        if(d not in matched_indices[:,0]):
            unmatched_detections.append(d)
    unmatched_trackers = []
    for t,trk in enumerate(trackers):
        if(t not in matched_indices[:,1]):
            unmatched_trackers.append(t)

    #filter out matched with low IOU
    matches = []
    for m in matched_indices:
        if(iou_matrix[m[0],m[1]]<iou_threshold):
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))
    if(len(matches)==0):
        matches = np.empty((0,2),dtype=int)
    else:
        matches = np.concatenate(matches,axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)



class Sort(object):
    def __init__(self,max_age=1,min_hits=3):
        """
        Sets key parameters for SORT
        """
        self.max_age = max_age
        self.min_hits = min_hits
        self.trackers = []
        self.frame_count = 0

    def update(self,dets):
        """
        Params:
        dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
        Requires: this method must be called once for each frame even with empty detections.
        Returns the a similar array, where the last column is the object ID.

        NOTE: The number of objects returned may differ from the number of detections provided.
        """
        self.frame_count += 1
        #get predicted locations from existing trackers.
        trks = np.zeros((len(self.trackers),5))
        to_del = []
        ret = []
        for t,trk in enumerate(trks):
            pos = self.trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
            if(np.any(np.isnan(pos))):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)
        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks)

        #update matched trackers with assigned detections
        for t,trk in enumerate(self.trackers):
            if(t not in unmatched_trks):
                d = matched[np.where(matched[:,1]==t)[0],0]
                trk.update(dets[d,:][0])

        #create and initialise new trackers for unmatched detections
        for i in unmatched_dets:
            trk = KalmanBoxTracker(dets[i,:]) 
            self.trackers.append(trk)
        i = len(self.trackers)
        for trk in reversed(self.trackers):
            d = trk.get_state()[0]
            if((trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits)):
                ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
            i -= 1
            #remove dead tracklet
            if(trk.time_since_update > self.max_age):
                self.trackers.pop(i)
        if(len(ret)>0):
            return np.concatenate(ret)
        return np.empty((0,5))

## Single Image

In [ ]:
f = frame()
tracker = Sort()
f.input_img = cv2.imread('test_images/test1.jpg')
f.result = tfnet.return_predict(f.input_img)
print(f.result)

In [ ]:
get_YOLO_detections(f)
update_detection_filter(f, tracker)
draw_detections(f, thick=6)

cv2.imshow("image", f.output_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Video Pipeline

In [28]:
# initialize
f = frame()
tracker = Sort(max_age=1, min_hits=3)
visualize = True
# read video file
cap = cv2.VideoCapture("project_video.mp4")
# set up video writer
shape_img = cap.read()[1].shape[1::-1]
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter("output_videos\\project_video_1.mp4",fourcc, 20.0, shape_img)
# process video
while(cap.isOpened()):
    ff,f.input_img = cap.read()
    if(ff == True):
        # obtain YOLO result
        f.result = tfnet.return_predict(f.input_img)
        # get detections
        get_YOLO_detections(f)
        # update filter
        update_detection_filter(f, tracker)
        # draw bounding boxes
        draw_detections(f, thick=6)
        # save output
        out.write(f.output_img)
        # visualize
        if(visualize == True):
            cv2.imshow('frame',f.output_img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()